In [3]:
import pandas as pd
from pandas_jsonlines.read_jsonlines import read_jsonlines
import requests
import urllib.request
import os
import re
from pdfminer.high_level import extract_text
from bs4 import BeautifulSoup
import json

In [8]:
###not sure if there are imports I need here or not

def details_new(strips):
    details = {}
    caseid = strips.index('Case summary')
    date = strips.index('Judgment date')
    cit = strips.index('Neutral citation')
    justices = strips.index('Justices')
    pdf = strips.index('Judgment (PDF)')
    details['Name'] = strips[0].replace('- The Supreme Court', '')
    if 'Case ID:' in strips[caseid-1] and 'UKSC' in strips[caseid-1]:
        details['Case ID'] = strips[caseid-1].replace('Case ID: ', '')
    elif 'Case ID:' in strips[caseid-1]:
        details['Case ID'] = strips[caseid-1].replace('Case ID:', 'UKSC')
    else:
        details['Case ID'] = strips[caseid-1]
    details['Judgment date'] = strips[date+1]
    details['Neutral citation'] = ', '.join(strips[cit+1:pdf])
    details['Justices'] = strips[justices+1]
    return details

def details_old(strips):
    details = {}
    if 'Neutral citation number' in strips:
        cit = strips.index('Neutral citation number')
    else:
        cit = strips.index('Neutral citation number(s)')
    date = strips.index('Judgment date')
    caseid = strips.index('Case ID')
    justices = strips.index('Justices')
    details['Name'] = strips[0].replace('- The Supreme Court', '')
    details['Case ID'] = ', '.join(strips[caseid+1:justices])
    details['Judgment date'] = strips[date+1]
    details['Neutral citation'] = ', '.join(strips[cit+1:caseid])
    details['Justices'] = strips[justices+1]
    return details

def addl_details(strips):
    details = {}
    caseid = strips.index('Case summary')
    if 'Issue' in strips:
        issues = strips.index('Issue')
    else:
        issues = strips.index('Issue(s)')
    facts = strips.index('Facts')
    if 'Judgment appealed' in strips:
        prev = strips.index('Judgment appealed')
    else:
        prev = strips.index('Parties')
    start = strips.index('Hearing start date')
    finish = strips.index('Hearing finish date')
    if 'Case ID:' in strips[caseid-1] and 'UKSC' in strips[caseid-1]:
        details['Case ID'] = strips[caseid-1].replace('Case ID: ', '')
    elif 'Case ID:' in strips[caseid-1]:
        details['Case ID'] = strips[caseid-1].replace('Case ID:', 'UKSC')
    else:
        details['Case ID'] = strips[caseid-1]
    details['Issue'] = ' '.join(strips[issues+1:facts])
    details['Facts'] = ' '.join(strips[facts+1:prev])
    if 'Judgment appealed' in strips:
        details['Judgment appealed'] = strips[prev+1]
    else:
        pass
    details['Hearing start date'] = strips[start+1]
    details['Hearing finish date'] = strips[finish+1]
    return details

def alt_extract_details(case):
    url = 'https://www.supremecourt.uk/cases/uksc-2018-0091.html'
    html = requests.get(url).content
    soup = BeautifulSoup(html, 'html.parser')
    strips = list(soup.stripped_strings)
    if 'Facts' in strips:
        details = details_new(strips)
    else:
        details = details_old(strips)
    details['URL'] = url
    return details

def alt_extract_additional(case):
    url = 'https://www.supremecourt.uk/cases/uksc-2018-0091.html'
    html = requests.get(url).content
    soup = BeautifulSoup(html, 'html.parser')
    strips = list(soup.stripped_strings)
    if 'Facts' in strips:
        details = addl_details(strips)
    else:
        return 'No additional details available for this case'
    details['URL'] = url
    return details

def alt_extract_all(case):
    j = {'body':'text'}
    ps = {'press summary':'text','other':'text'}
    d = alt_extract_details(case)
    ad = alt_extract_additional(case)
    return j, ps, d, ad

def parse_all(judgement, summary, details, addl):
    j = judgement
    ps = summary
    d = details
    ad = addl
    return {'judgement':j, 'press summary':ps, 'details':d, 'additional':ad}

In [9]:
alt_extract_all('uksc-2018-0129')

({'body': 'text'},
 {'press summary': 'text', 'other': 'text'},
 {'Name': 'R v Mackinlay and others (Respondents) ',
  'Case ID': 'UKSC 2018/0091',
  'Judgment date': '25 Jul 2018',
  'Neutral citation': '[2018] UKSC 42',
  'Justices': 'Lady Hale, Lord Mance, Lord Hughes, Lord Hodge, Lord Lloyd-Jones',
  'URL': 'https://www.supremecourt.uk/cases/uksc-2018-0091.html'},
 'No additional details available for this case')

In [2]:
parse_all({'body': 'text'},
 {'press summary': 'text', 'other': 'text'},
 {'Name': 'R v Mackinlay and others (Respondents) ',
  'Case ID': 'UKSC 2018/0091',
  'Judgment date': '25 Jul 2018',
  'Neutral citation': '[2018] UKSC 42',
  'Justices': 'Lady Hale, Lord Mance, Lord Hughes, Lord Hodge, Lord Lloyd-Jones',
  'URL': 'https://www.supremecourt.uk/cases/uksc-2018-0091.html'},
 'No additional details available for this case')

NameError: name 'parse_all' is not defined